# Integrate ode and approximate with a network

In this notebook we create a dataset from a differential equation and then train an approximate neural network on that data.

In [0]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


## define differential equations



###  Lorenz 63 model

$dx/dt  = - \sigma (y-x)$

$dy/dt = x (\rho -z) -y$

$dz/dt = x y - \beta z$

$x(0)= x_0, y(0)=y_0, z(0)=z_0$

In the code we'll use x to denote a vector with (x,y,z)
See also [Lorenz at wikipedia](https://en.wikipedia.org/wiki/Lorenz_system)

In [0]:
# lorenz 63 equations
sigma = 10.0
beta = 8.0/3.0
rho=28.0
def lorenz63(t,state):
    x, y, z = state
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z 
x0_true = [-1.46938568, -0.82968588, 20.33234264]

## Simulate model

In [0]:
tstart=0.0
tend=40.0
tstep=0.1
t_output=np.arange(tstart,tend,step=tstep)
sol = solve_ivp(lorenz63, [tstart, tend], x0_true, t_eval=t_output)

fig, axs = plt.subplots(3, figsize=(15,4))
fig.suptitle('Vertically stacked subplots')
axs[0].plot(sol.t,sol.y[0])
axs[0].set_ylabel("x")
axs[1].plot(sol.t,sol.y[1])
axs[1].set_ylabel("y")
axs[2].plot(sol.t,sol.y[2])
axs[2].set_ylabel("z")
axs[2].set_xlabel("t")
plt.show()

In [0]:
# create training dataset
tstart=0.0
tend=1000.0
tstep=0.1
t_train=np.arange(tstart,tend,step=tstep)
x0_train= [-10.41904706, -16.07509044,  20.99289982]
sol_train = solve_ivp(lorenz63, [tstart, tend], x0_train, t_eval=t_train)
nt_train=len(t_train)
x_train = sol_train.y.transpose()
y_train = np.zeros(sol_train.y.shape)
for i in range(nt_train):
    y_train[:,i]=lorenz63(sol_train.t[i],sol_train.y[:,i])
y_train = y_train.transpose()

# and a test dataset
x0_test=[12.05802086, 14.7646725 , 28.37399324]
t_test=np.arange(tstart,tend,step=tstep)
sol_test = solve_ivp(lorenz63, [tstart, tend], x0_test, t_eval=t_test)

nt_test=len(t_test)
x_test = sol_test.y.transpose()
y_test = np.zeros(sol_test.y.shape)
for i in range(nt_test):
    y_test[:,i]=lorenz63(sol_test.t[i],sol_test.y[:,i])
y_test = y_test.transpose()


In [0]:
xt_mean = x_train.mean(axis=0)
xt_std  = x_train.std(axis=0)
x_train = (x_train - xt_mean)/xt_std
yt_mean = y_train.mean(axis=0)
yt_std  = y_train.std(axis=0)
y_train = 0.5+0.2*(y_train - yt_mean)/yt_std

xs_mean = x_test.mean(axis=0)
xs_std  = x_test.std(axis=0)
x_test  = (x_test - xs_mean)/xs_std
ys_mean = y_test.mean(axis=0)
ys_std  = y_test.std(axis=0)
y_test = 0.5+0.2*(y_test - ys_mean)/ys_std


In [0]:
y_train

## Train a Neural Net on the generated data



In [0]:
input_shape = (3)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="sigmoid"),
        layers.Dense(20, activation="sigmoid"),
        layers.Dense(3, activation="sigmoid")
    ]
)

model.summary()

In [0]:
batch_size = 200
epochs = 15

#model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## Evaluate for test data


In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [0]:
?keras.optimizers.Adam